In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/Users/cslage/Research/LSST/optical_simulator/CTE/'
%cd $thedir

#%matplotlib inline

In [ ]:
#files = glob.glob('ITL-3800C-029_flat_light_004_20161010??????.fits')
files = sort(glob.glob('ITL-3800C-029_flat_light_015_20161017??????.fits'))
name = files[0][25:28]
print files
print name

In [ ]:
xstart = 50
xstop = 450
ov_start = 2000
ystart = 2000
ystop = 2022
xovstart = 520
xovstop = 542
num_last = 4

for file in files:
    name = file[25:28]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    figure(figsize=(15,15))
    subplots_adjust(hspace = 0.5, wspace = 0.5)
    suptitle("Parallel CTE by segment - ITL CCD 029 - %s"%name, fontsize = 16)

    for segment in range(16):
                xaxis = []

                subplot(4,4,segment+1)
                imhdr=hdulist[segment+1].header
                extname = imhdr['EXTNAME']
                title("CTE Delay - %s"%extname)
                data = array(hdulist[segment+1].data + 32768, dtype = int32)
                flat_overscan = array(data[ystop-num_last:ystop,xovstart:xovstop].sum() / float(num_last*(xovstop-xovstart)))
                
                #print (data - flat_overscan)[ystart:ystop, 256]
                #sys.exit()
                
                cte_data = (data - flat_overscan)[:,xstart:xstop].mean(axis=1)
                for i in range(len(cte_data)):
                    xaxis.append((i+1)*2.6E-6*542)
                slope, intercept, r_value, p_value, std_err = stats.linregress(xaxis[ystop-10:ystop], cte_data[ystop-10:ystop])
                x_intercept = -intercept / slope
                #xplot=linspace(0.0,7.0,100)
                yplot = slope * array(xaxis) + intercept
                scatter(xaxis[ystart:ystop], cte_data[ystart:ystop], color = 'blue')
                plot(xaxis[ystart-5:ystop], yplot[ystart-5:ystop], ls = "--", lw = 4, color = 'red')
                #errorbar(xaxis, cte_data, yerr=cte_std)
                ytext = cte_data[ystart]*1.1
                text(2.802, ytext, "Intercept = %.2f seconds"%x_intercept, fontsize = 12)
                #yscale('log',nonposy='clip')
                #ylim(.01, 100000)
                ylim(0,ytext*1.1)
                #xlim(2.5,3.5)
                xticks([2.80, 2.85])
                xlabel("Time(seconds)")
                ylabel("Signal(ADU)")
                print "Segment %s done"%extname
    #show()
    savefig("Parallel_CTE_Delay_%s_25Oct16.pdf"%name)
    #savefig("Parallel_CTE_Lin_%s_18Oct16.pdf"%name)
    #clf()